In [ ]:
import torch.nn as nn
import torch.optim as optim

from lstm import LSTM
from preprocessing import *
from utils import *
from features import *

if torch.backends.mps.is_available():
    device = torch.device("mps")
    torch.mps.set_per_process_memory_fraction(0.)
else:
    device = torch.device("cpu")
device

In [ ]:
raw_data = pd.read_csv('consumption_and_temperatures.csv')
raw_data['timestamp'] = pd.to_datetime(raw_data['timestamp'])

seq_len = 72
scale_output=True
target_column = 'NO1_consumption'

features_to_add = [
    (   
        pick_location_data,
        { 'loc': [1] }
    ),
    (
        add_season_columns, 
        {}
    ),
    (
        shift_data, 
        {
            "shift_max": 10,
            "column_to_shift": "NO1_temperature",
            "new_column_name": "temp"
        }
    ),
    (
        add_hour_columns,
        {}
    ),
    (
        previous_y,
        {'target': target_column}
    )
]


In [ ]:
(X_train, y_train), (X_val, y_val), (X_test, y_test), (scalerInputMethod, scalerOutputMethod), df_target = general_preprocessing(
        raw_data, 
        features_to_add=features_to_add,
        seq_len=seq_len,
        scale_output=scale_output
    )

input_size = X_train.shape[2]
hidden_size = 11
num_layers = 3

model = LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)
model.to(device)

lr = 1e-3
num_epochs=100

loss_func = nn.MSELoss()
opt = optim.Adam(model.parameters(), lr=lr)

path2 = "LSTM-prev-y-2024-03-20-loss-0.04269014"
path2 = "models/" + path2
model.load_state_dict(torch.load(path2))
model.to(device)

model, losses, val_loss, opt = fit(
    model, 
    train_set=(X_train, y_train),
    val_set=(X_val, y_val),
    opt=opt,
    loss_func=loss_func,
    num_epochs=num_epochs
)

# path = f'models/LSTM-prev-y-{str(datetime.now().date())}-loss-{str(losses[len(losses)-1])}'
# torch.save(model.state_dict(), path)

plt.plot(losses, label="train loss")
plt.plot(val_loss, label="val loss")
plt.legend()

y_test, y_pred = predict(model, scalerOutputMethod, (X_test, y_test))
plot_error_by_hour_for_test_set(y_test, y_pred, start_hour=df_target['timestamp'].dt.hour.iloc[seq_len-1])
for k in range(5):
    make_forecast_replace_previous_y(model, scalerOutputMethod, (X_test, y_test), seq_len=seq_len)